### <center> **Hyperparameters**

In [ ]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики
from sklearn import ensemble #ансамбли
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
import optuna
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import networkx as nx


In [ ]:
#чтение файла
data = pd.read_csv('/Users/dariashavronskaia/Desktop/data science/data/_train_sem09.csv') 

In [ ]:
#посмотрим на данные
data.head()

In [ ]:
y = data['Activity']
X = data.drop(columns='Activity')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

#### <center> **Logistic Regression**

In [157]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


#### <center> **Random Forest**

In [158]:
#Создаем объект класса логистическая регрессия
rf = ensemble.RandomForestClassifier(random_state=42)
#Обучаем модель, минимизируя logloss
rf.fit(X_train, y_train)
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


### <center> **GridSearchCV**

#### <center> **Logistic Regression**

In [159]:
#пропишем параметры
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=1, max_iter=1000), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
grid_search_1.fit(X_train, y_train) 
y_test_pred = grid_search_1.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.p

f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


In [160]:
print("Наилучшая модель:\n{}".format(grid_search_1.best_estimator_))
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(grid_search_1.best_score_))

Наилучшая модель:
LogisticRegression(C=0.1, max_iter=1000, random_state=1, solver='liblinear')
Наилучшее значение точности при кросс-валидации: 0.76


#### <center> **Random Forest**

In [161]:
# Определение сетки параметров для случайного леса
param_grid_rf = {
    'n_estimators': [100, 150, 200],  # Количество деревьев в лесу
    'max_depth': [10, 20, 30],  # Максимальная глубина деревьев
    'min_samples_leaf': [1, 2, 4]  # Минимальное количество примеров в листе
}

# Инициализация GridSearchCV для случайного леса
grid_search_rf = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=1), 
    param_grid=param_grid_rf, 
    cv=5, 
    n_jobs=-1
)

# Обучение модели на обучающих данных
grid_search_rf.fit(X_train, y_train)

# Предсказание на тестовом наборе данных
y_test_pred_rf = grid_search_rf.predict(X_test)

# Вывод метрики f1_score на тестовом наборе
print('f1_score на тестовом наборе для случайного леса: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

# Вывод наилучших значений гиперпараметров для случайного леса
print("Наилучшие значения гиперпараметров для случайного леса: {}".format(grid_search_rf.best_params_))

f1_score на тестовом наборе для случайного леса: 0.83
Наилучшие значения гиперпараметров для случайного леса: {'max_depth': 30, 'min_samples_leaf': 2, 'n_estimators': 100}


In [162]:
print("Наилучшая модель:\n{}".format(grid_search_rf.best_estimator_))
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(grid_search_rf.best_score_))

Наилучшая модель:
RandomForestClassifier(max_depth=30, min_samples_leaf=2, random_state=1)
Наилучшее значение точности при кросс-валидации: 0.80


### <center> **RandomizedSearchCV**

#### <center> **Logistic Regression**

In [163]:
#пропишем параметры
param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))}
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  

random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None

f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}


In [164]:
print("Наилучшая модель:\n{}".format(random_search.best_estimator_))
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(random_search.best_score_))

Наилучшая модель:
LogisticRegression(C=0.01, max_iter=1000, random_state=42)
Наилучшее значение точности при кросс-валидации: 0.75


#### <center> **Random Forest**

In [168]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
grid_search_forest.fit(X_train, y_train) 
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}


In [166]:
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(grid_search_forest.best_score_))

Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}
Наилучшее значение точности при кросс-валидации: 0.79


### <center> **Hyperopt**

#### <center> **Logistic Regression**

In [ ]:
print("Версия Hyperopt : {}".format(hyperopt.__version__))

In [ ]:
space = {'penalty': hp.choice('penalty', ['l1', 'l2']), # тип регуляризации
       'solver': hp.choice('solver', ['liblinear', 'saga']), # алгоритм оптимизации
       'C': hp.uniform('C', low=0.01, high=1) # уровень силы регурялизации
       }

In [ ]:
random_state = 42

def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {
        'penalty': params['penalty'], # тип регуляризации
        'solver': params['solver'], # алгоритм оптимизации
        'C': params['C'] # уровень силы регурялизации
    }
    model = linear_model.LogisticRegression(**params, random_state=random_state)
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    return {'loss': -score, 'params': params, 'status': STATUS_OK}

In [ ]:
trials = Trials()

best = fmin(hyperopt_lr,
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )

In [ ]:
print("Наилучшие значения гиперпараметров {}".format(trials.best_trial['result']['params']))

In [ ]:
model = linear_model.LogisticRegression(
    random_state=random_state, 
    penalty=trials.best_trial['result']['params']['penalty'],
    solver=trials.best_trial['result']['params']['solver'],
    C=float(trials.best_trial['result']['params']['C'])
)

model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

#### <center> **Random Forest**

In [ ]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [ ]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [ ]:
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

In [ ]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

## <center> Optuna

In [169]:
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.6.1


#### <center> **Logistic Regression**

In [170]:
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l2', 'none'])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'sag'])
  C = trial.suggest_float('C', 0.01, 1, step= 5)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
    
  scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')

  return np.mean(scores)
  
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

[I 2024-04-11 14:47:23,835] A new study created in memory with name: LogisticRegression
/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/optuna/distributions.py:685: UserWarning:

The distribution is specified by [0.01, 1] and step=5, but the range is not divisible by `step`. It will be replaced by [0.01, 0.01].

/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning:

`penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.

/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning:

Setting penalty=None will ignore the C and l1_ratio parameters

/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/

KeyboardInterrupt: 

In [ ]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

#### <center> **Random Forest**

In [ ]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')

  return np.mean(scores)

# cоздаем объект исследования
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

In [ ]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

In [ ]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1 score - это метрика, используемая в задачах классификации, которая учитывает как точность (precision), так и полноту (recall) модели. Он является гармоническим средним между точностью и полнотой и позволяет оценить баланс между ними.
F1 score должен быть как можно ближе к 1, что означает более высокую точность и полноту модели. Он является хорошим показателем для сравнения моделей классификации, особенно когда классы несбалансированы.
Нам удалось увеличить F1 score при подборе гиперпараметров c кросс-валидацией. 